In [11]:
import pandas as pd
import numpy as np
import csv
from tqdm.notebook import tqdm
from multiprocessing import Process

In [2]:
def distance_from_lonlat(p1,p2):
    # https://github.com/openlayers/openlayers/blob/release-2.12/lib/OpenLayers/Util.js#L750
    #VincentyConstants
    a = 6378137
    b = 6356752.3142
    f = 1/298.257223563
    
    p1_lon = (np.pi/180) * p1[0]
    p1_lat = (np.pi/180) * p1[1]
    p2_lon = (np.pi/180) * p2[0]
    p2_lat = (np.pi/180) * p2[1]
    
    L = p2_lon - p1_lon
    U1 = np.arctan((1-f) * np.tan(p1_lat))
    U2 = np.arctan((1-f) * np.tan(p2_lat))
    cosU1 = np.cos(U1)
    cosU2 = np.cos(U2)
    sinU1 = np.sin(U1)
    sinU2 = np.sin(U2)
    
    
    lambda_ = L
    lambdaP = 2*np.pi
    iterLimit = 30
    
    while(np.abs(lambda_ - lambdaP) > 1e-13):
        sinSigma = np.sqrt((cosU2*np.sin(lambda_))**2 + (cosU1*sinU2- sinU1*cosU2*np.cos(lambda_))**2)
        if (sinSigma == 0):
            return 0
        cosSigma = sinU1*sinU2 + cosU1*cosU2*np.cos(lambda_)
        sigma = np.arctan2(sinSigma, cosSigma)
        alpha = np.arcsin(cosU1*cosU2*np.sin(lambda_)/sinSigma)
        cosalpha = np.cos(alpha)
        sinalpha = np.sin(alpha)
        CSM   = cosSigma - 2*sinU1*sinU2/(cosalpha**2)
        C     = (f/16)*cosalpha**2*(4+f*(4-3*(cosalpha**2)))
        lamdaP = lambda_
        lambda_ = L + (1-C)*f*sinalpha*(sigma + C*sinSigma*(CSM + C*cosSigma*(-1 +2*CSM**2)))
        iterLimit -= 1
        if iterLimit <= 0:
                        break
    uSq = cosalpha*(a**2 - b **2)/(b**2)
    A = 1 + uSq/16384*(4096+uSq*(-768+uSq*(320-175*uSq)))
    B = uSq/1024 * (256+uSq*(-128+uSq*(74-47*uSq)))
    deltaSigma = B*sinSigma*(CSM+B/4*(cosSigma*(-1+2*CSM**2)-
    B/6*CSM*(-3+4*sinSigma*sinSigma)*(-3+4*CSM**2)))
    s = b*A*(sigma-deltaSigma)
    return s

In [3]:
bus_lat_list = []
bus_lon_list = []
with open('bus_stop_coordinates.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        bus_lat_list.append(float(row[0]))
        bus_lon_list.append(float(row[1]))

zipped_bus = list(zip(bus_lon_list,bus_lat_list))


df = pd.read_csv("Data_Lv3_UMDOTS_Escooters.csv")


lon_list_start = list(map(float,df['START LONG'][:]))
lat_list_start = list(map(float,df['START LAT'][:])) 
zipped_start = list(zip(lon_list_start,lat_list_start))

lon_list_end = list(map(float,df['END LONG'][:]))
lat_list_end = list(map(float,df['END LAT'][:]))
zipped_end = list(zip(lon_list_end,lat_list_end))

In [4]:
radius = 50

thread_1_list = []
thread_2_list = []
thread_3_list = []
thread_4_list = []

for i in tqdm(range(len(zipped_bus))):
    for j in tqdm(range(len(zipped_bus)),leave=False):
        for k in range(len(zipped_start)):
            start = zipped_start[k]
            if (distance_from_lonlat(zipped_bus[i],start) < radius):
                end   = zipped_end[k] 
                if((distance_from_lonlat(zipped_bus[j],end) < radius)):
                    thread_1_list.append([i,j,df['MINUTES'][k]])




  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

...

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

In [5]:
with open('rides_between_bus_stops.csv','w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Start_idx', 'End_idx', 'Duration'])
    for row in thread_1_list:
        writer.writerow(row)


In [10]:
distinct_stops = [] 
for ride in thread_1_list:
    if not (ride[0] == ride[1]):
        distinct_stops.append(ride)
    

In [12]:
with open('rides_between__distinct_bus_stops.csv','w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Start_idx', 'End_idx', 'Duration'])
    for row in distinct_stops:
        writer.writerow(row)



In [4]:
import csv

distinct_stops = []
with open('rides_between__distinct_bus_stops.csv') as csvfile:
    reader=  csv.reader(csvfile,delimiter=',')
    for row in reader:
        distinct_stops.append(row)
        
    

['Start_idx', 'End_idx', 'Duration']

In [28]:
ODTable = np.zeros([22,22,2])
for entry in distinct_stops:
    indices = list(map(int,entry[:2]))
    row = min(indices)
    column = max(indices)
    ODTable[row,column,0] += int(entry[2])
    ODTable[row,column,1] += 1
    
for i in range(22):
    for j in range(22):
        if (ODTable[i,j,1] > 0):
            ODTable[i,j,0] = ODTable[i,j,0]/ODTable[i,j,1] 
    
    
            

In [19]:
min(list(map(int,distinct_stops[0][:2])))

0

In [50]:
print(ODTable[20])

[[0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [4.5 2. ]]
